## Introduction

* The code in this repository implements the [Double DQN algorithm](https://arxiv.org/abs/1509.06461).

## Q-Function Approximation
* The Q-function update in standard Q learning is
\begin{align}
Q(s_t,a_t) &\leftarrow Q(s_t,a_t) + \alpha(r_{t+1} + \gamma\max_{a}Q(s_{t+1},a) - Q(s_t,a_t))\\
\Delta(s_t, a_t) &= \alpha(r_{t+1} + \gamma\max_{a}Q(s_{t+1},a) - Q(s_t,a_t))
\end{align}
where
   * $s_t$ is the state at time $t$,
   * $a_t$ is the action taken at time $t$,
   * $r_{t+1}$ is the reward obtained, 
   * $s_{t+1}$ is the state at $t+1$, 
   * $\gamma \in (0,1]$ is the discount factor
   * $\alpha \in (0,1]$ is the step size.


* In the  DQN (Deep Q-network) algorithm, Q values are estimated using neural networks, and the update rule becomes 
  \begin{align}
   \Delta(s_t, a_t, w) &= \alpha(r_{t+1} + \gamma\max_{a}\hat{Q}(s_{t+1},a,w) - \hat{Q}(s_t,a_t,w))
  \end{align}
  where 
    * $\hat{Q}$ is the neural network approximation to the Q-function called the local neural network, and
    * $w$ are the weights of the local neural network.

* DQN makes the following modification in order to train the local neural network in a stable manner:
  \begin{align}
    \Delta(s_t, a_t, w, \bar{w}) &= \alpha(r_{t+1} + \gamma\max_{a}\hat{Q}(s_{t+1},a,\bar{w}) - \hat{Q}(s_t,a_t,w))
  \end{align}
  where 
   * $\bar{w}$ are the weights of the target neural network. 

* The $\bar{w}$ are obtained as follows:
  \begin{align}   
     \bar{w}(t) &= w(n\cdot T)\;\; \forall t\; \in  \lbrack n \cdot T,(n+1)\cdot T)  
  \end{align}  
  where 
   * $n$ is a non-negative integer,  
   * $T$ is the update period an is a positive integer.

* Instead of keeping the target neural network constant for a duration $T$, one can update the target neural network at every update of the local neural network as follows:
\begin{align}
\bar{w} &\leftarrow \tau w  + (1 -\tau) \bar{w} 
\end{align}
where $\tau \in [0,1]$.  The weighted sum allows $\bar{w}$ to change relatively more slowly  for values of $\tau$ closer to $0$.

* The double DQN algorithm tries to prevent the overestimation of the Q-function using both the target and local network weights in computing the next state's Q-function value;
  \begin{align}
    \Delta(s_t, a_t, w, \bar{w}) &= \alpha(r_{t+1} + \gamma \hat{Q}(s_{t+1},\mathop{argmax}_a \hat{Q}(s_{t+1}, a, w),\bar{w}) - \hat{Q}(s_t,a_t,w))
  \end{align} where  $r_{t+1} + \gamma \hat{Q}(s_{t+1},\mathop{argmax}_a \hat{Q}(s_{t+1}, a, w),\bar{w})$ is the TD target.

## Target Neural Network Training
* In order to train the target neural network, states and rewards are used.
* At  every time step $t$, the following quantities are stored for training;
    * $s_t$: the current state,
    * $a_t$: the current action,
    * $r_{t+1}$: the reward of the current action
    * $s_{t+1}$: the next state.



* The set $\{s_t, a_t, r_{t+1}, s_{t+1}\}$ is called an experience.
* Experiences are saved in a buffer called the experience replay buffer.
* The experiences can be stored in a circular buffer of size $B$ where $B \geq N$. 
* At every training step, a batch of $N$ experiences are randomly selected to update the local network weights.
* The local neural network is trained to minimize the root mean squared difference between the TD target and the estimate $\hat{Q}(s_t,a_t,w)$.
* The local neural network weights are updated every $T$ time steps:
\begin{align}
  y_i    &= r_{t+1, i} + \gamma \hat{Q}(s_{t+1,i},\mathop{argmax}_a \hat{Q}(s_{t+1,i}, a, w),\bar{w})\;\; \forall i \in \{1, \dots, N\}\\
  \hat{y}_i & = \hat{Q}(s_{t,i}, a_{t,i}, w)\;\; \forall; i \in \{1, \dots, N\} \\
\Delta w &= \sum_{i=1}^{N}\alpha( y_i - \hat{y}_i) \nabla  \hat{Q}_w(s_{t,i}, a_{t,i}, w)\\
w &\leftarrow w + \Delta w
\end{align}
* The target neural network weights are updated using
\begin{align}
\bar{w} &\leftarrow \tau w  + (1 -\tau) \bar{w} 
\end{align}


##  Training Algorithm

1. Set the following hyper parameters:
   * $\epsilon$: the frequency of random action choices, $\epsilon \in [0,1]$;
   * $\gamma_\epsilon$: decay rate for $\epsilon$, $\gamma_\epsilon \in [0,1]$;
   * $\epsilon_{min}$: lower bound on $\epsilon$
   * $\alpha$: the learning rate, $\alpha \in (0,1]$;
   * $\gamma$: the discount factor, $\gamma \in (0,1]$;
   * $\tau$: target weight update factor, $\tau in (0,1]$;
   * $T$: period of local network update, $T > 0$;
   * $N$: training batch size, $N > 0$;
   * $B$: circular buffer capacity, $B \leq >N$.
   * $N_e$: set the maximum number of episodes used in training.
   
2. Determine the local and target neural network structure. 
   1.  Initialize $w$ and $\bar{w}$, $\bar{w} = w$.
3. Perform training:
   1. Start new episode while number of episodes is less than $N_e$
      1. Choose action $a_t$ using  $\hat{Q}$ and the $\epsilon$-Greedy algorithm.
      2. Take the action $a_t$, observe the reward $r_{t+1}$, transition to next state $s_{t+1}$. 
      3. Store the experience $\{s_t, a_t, r_{t+1}, s_{t+1}\}$ in the circular  buffer
      4. If $t = nT$ for some positive integer $n$ then update the target and local neural network weights as follows
          1. Get $N$ random experiences from the replay buffer. 
          2. Update the local network weight $w$:
             \begin{align}
                y_i    &= r_{t+1, i} + \gamma \hat{Q}(s_{t+1,i},\mathop{argmax}_a \hat{Q}(s_{t+1,i}, a, w),\bar{w})\;\;  \forall i \in \{1, \dots, N\}\\
             \hat{y}_i & = \hat{Q}(s_{t,i}, a_{t,i}, w)\;\; \forall i \in \{1,\dots,N\} \\
             \Delta w &= \sum_{i=1}^{N}\alpha( y_i - \hat{y}_i) \nabla  \hat{Q}_w(s_{t,i}, a_{t,i}, w)\\
              w &\leftarrow w + \Delta w
             \end{align}
          3. Update the target network weight
             \begin{align}
                \bar{w} &\leftarrow \tau w  + (1 -\tau) \bar{w} 
            \end{align}
      5. Update $\epsilon$:
         \begin{align*}
           \epsilon &\leftarrow \max(\epsilon_{min}, \gamma_{epsilon}\epsilon)
         \end{align*}
         


## Hyperparameters

* The hyperparameters used in the training are:
   * $\epsilon=1.0$
   * $\gamma_\epsilon=0.8$
   * $\epsilon_{min}= 0.0001$
   * $\alpha=0.001$
   * $\gamma=0.99$
   * $\tau = 0.01$
   * $T =4$
   * $N = 256$ 
   * $B = 10000$.
   * $N_e = 1800$.

* The hyperparameters are determined by trial and error.

# Local and Target Neural Network Structure
* The structure of the local and target neural networks are the same.
    * The input layers have 37 nodes each corresponding to the dimension of the signal received by the environment.
    * The outputs of the input layer nodes  are fed to RELU activation functions.
    * Each network has 2 hidden layers with 64 nodes in each layer.
    * The hidden layer nodes' outputs are fed to RELU activation functions.
    * The output layers have 4 nodes each corresponding to the number of available actions. 
* The initial weights $w$ consists of random numbers.
* The Adam optimization algorithm is used to determine $\Delta w$. 





# Training Results
* The agent achieves an average score of 13 or above in about 360 episodes.
* The progression of the scores and the average score is shown in the next figure.
![Training Performance](training_performance.png)


## Future Enhancements
* The hyper parameters for the algorithm	have been chosen by trial and error. Better parameters can be found by using an	automated hyperparameter search	techniques.
* Intuitively,	 increasing the number of training	samples	should improve the time performance, but would it woul also increase the training time. [Prioritized experienced replay](https://arxiv.org/abs/1511.05952) can improve the performance with less	increase in training time. Here, the training samples are given priorities based on	their TD error.

* Finally, there are many improvements to the vanilla DQN algorithm. One can combine all into to obtain	the [Rainbow algorithm](https://arxiv.org/abs/1710.02298).
* One open question is how to reduce the variance in the scores. The average score is improved as a result of training, but the scores vary a lot around this average value.